In [86]:
%%writefile train.py
# train_MOLANE.py - Training U-Net su MoLane

import os
import warnings
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torch.amp import autocast, GradScaler
import segmentation_models_pytorch as smp
from tqdm import tqdm
import numpy as np

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', message='.*UnsupportedFieldAttributeWarning.*')

from dataset import LaneSegmentationDataset
from augmentation import (
    get_training_augmentation_improved,
    get_validation_augmentation_improved,
)
from metrics import LaneMetrics

# ==================== LOSS FUNCTIONS ====================

class FocalTverskyLoss(nn.Module):
    def __init__(self, alpha=0.3, beta=0.7, gamma=0.75, smooth=1.0):
        super().__init__()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.smooth = smooth
    
    def forward(self, predictions, targets):
        probs = torch.sigmoid(predictions)
        TP = (probs * targets).sum()
        FP = (probs * (1 - targets)).sum()
        FN = ((1 - probs) * targets).sum()
        
        tversky_index = (TP + self.smooth) / (
            TP + self.alpha * FP + self.beta * FN + self.smooth
        )
        
        focal_tversky = torch.pow(1 - tversky_index, self.gamma)
        return focal_tversky


class CombinedLossOptimized(nn.Module):
    def __init__(self, focal_weight=0.6, dice_weight=0.4):
        super().__init__()
        self.focal_weight = focal_weight
        self.dice_weight = dice_weight
        self.focal_tversky = FocalTverskyLoss(alpha=0.3, beta=0.7, gamma=0.75)
    
    def forward(self, predictions, targets):
        focal = self.focal_tversky(predictions, targets)
        
        probs = torch.sigmoid(predictions)
        intersection = (probs * targets).sum()
        dice = (2.0 * intersection + 1.0) / (probs.sum() + targets.sum() + 1.0)
        dice_loss = 1 - dice
        
        return self.focal_weight * focal + self.dice_weight * dice_loss


class WarmupCosineScheduler:
    def __init__(self, optimizer, warmup_epochs, max_epochs, min_lr=1e-6):
        self.optimizer = optimizer
        self.warmup_epochs = warmup_epochs
        self.max_epochs = max_epochs
        self.min_lr = min_lr
        self.base_lr = optimizer.defaults['lr']
    
    def step(self, epoch):
        if epoch < self.warmup_epochs:
            lr = self.base_lr * (epoch + 1) / self.warmup_epochs
        else:
            lr = self.min_lr + (self.base_lr - self.min_lr) * \
                 0.5 * (1 + np.cos(np.pi * (epoch - self.warmup_epochs) / (self.max_epochs - self.warmup_epochs)))
        
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        
        return lr


# ==================== CONFIGURAZIONE MOLANE ====================

class Config:
    # 🎯 MOLANE PATHS
    MOLANE_BASE = '/kaggle/input/carlane-benchmark/CARLANE/MoLane/data'
    
    # Validation set
    VAL_IMAGES_DIR = os.path.join(MOLANE_BASE, 'val', 'target')
    VAL_MASKS_DIR = '/kaggle/working/molane_val_masks'
    
    # Test set
    TEST_IMAGES_DIR = os.path.join(MOLANE_BASE, 'test', 'target')
    TEST_MASKS_DIR = '/kaggle/working/molane_test_masks'
    
    # Parametri training
    TRAIN_RATIO = 0.8
    VAL_RATIO = 0.2
    
    ENCODER = 'efficientnet-b4'
    ENCODER_WEIGHTS = 'imagenet'
    CLASSES = 1
    ACTIVATION = None
    
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    EPOCHS = 30
    BATCH_SIZE = 16
    LEARNING_RATE = 5e-4
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.2
    NUM_WORKERS = 4
    
    USE_AMP = True
    WARMUP_EPOCHS = 3
    
    MODEL_DIR = '/kaggle/working/models'
    BEST_MODEL_PATH = os.path.join(MODEL_DIR, 'best_unet_molane.pth')
    LAST_MODEL_PATH = os.path.join(MODEL_DIR, 'last_unet_molane.pth')


# ==================== DATASET WRAPPERS ====================

class TrainAugmentedDataset:
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image, mask = self.dataset[idx]
        if self.transform:
            sample = self.transform(image=image, mask=mask)
            image = sample['image']
            mask = sample['mask']
        return image, mask


class ValAugmentedDataset:
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image, mask = self.dataset[idx]
        if self.transform:
            sample = self.transform(image=image, mask=mask)
            image = sample['image']
            mask = sample['mask']
        return image, mask


# ==================== TRAINING FUNCTIONS ====================

def create_model(config):
    """Crea il modello U-Net"""
    return smp.Unet(
        encoder_name=config.ENCODER,
        encoder_weights=config.ENCODER_WEIGHTS,
        classes=config.CLASSES,
        activation=config.ACTIVATION,
        decoder_dropout=config.DROPOUT,
    )


def create_dataloaders_molane(config):
    """Crea train/val dataloaders da MoLane"""
    
    print("📂 Caricamento dataset MoLane (val + test)...")
    
    # Carica val + test
    val_dataset = LaneSegmentationDataset(
        images_dir=config.VAL_IMAGES_DIR,
        masks_dir=config.VAL_MASKS_DIR,
        transform=None,
        preprocessing=None,
    )
    
    test_dataset = LaneSegmentationDataset(
        images_dir=config.TEST_IMAGES_DIR,
        masks_dir=config.TEST_MASKS_DIR,
        transform=None,
        preprocessing=None,
    )
    
    full_dataset = ConcatDataset([val_dataset, test_dataset])
    
    total_size = len(full_dataset)
    print(f"✅ Dataset MoLane caricato: {total_size} immagini totali")
    
    train_size = int(config.TRAIN_RATIO * total_size)
    val_size = total_size - train_size
    
    print(f"\n📊 Split ratio: {config.TRAIN_RATIO*100:.1f}% training / {config.VAL_RATIO*100:.1f}% validation")
    print(f"   Train samples: {train_size}")
    print(f"   Val samples: {val_size}")
    
    train_dataset, val_dataset = random_split(
        full_dataset,
        [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    train_dataset_augmented = TrainAugmentedDataset(
        dataset=train_dataset,
        transform=get_training_augmentation_improved(),
    )
    
    val_dataset_augmented = ValAugmentedDataset(
        dataset=val_dataset,
        transform=get_validation_augmentation_improved(),
    )
    
    train_loader = DataLoader(
        train_dataset_augmented,
        batch_size=config.BATCH_SIZE,
        shuffle=True,
        num_workers=config.NUM_WORKERS,
        pin_memory=True,
        prefetch_factor=2,
    )
    
    val_loader = DataLoader(
        val_dataset_augmented,
        batch_size=config.BATCH_SIZE,
        shuffle=False,
        num_workers=config.NUM_WORKERS,
        pin_memory=True,
        prefetch_factor=2,
    )
    
    print(f"✅ DataLoaders creati!")
    return train_loader, val_loader


def train_epoch(model, train_loader, optimizer, loss_fn, device, scaler=None, use_amp=False):
    model.train()
    total_loss = 0
    
    pbar = tqdm(train_loader, desc="Training")
    
    for images, masks in pbar:
        images = images.to(device)
        masks = masks.unsqueeze(1).to(device)
        
        if use_amp and scaler:
            with autocast('cuda'):
                outputs = model(images)
                loss = loss_fn(outputs, masks)
            
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
            optimizer.step()
        
        optimizer.zero_grad()
        total_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.6f}'})
    
    return total_loss / len(train_loader)


def validate_epoch(model, val_loader, loss_fn, device, threshold=0.5):
    model.eval()
    total_loss = 0
    all_metrics = {'iou': [], 'dice': [], 'f1': []}
    
    with torch.no_grad():
        for images, masks in tqdm(val_loader, desc="Validating"):
            images = images.to(device)
            masks = masks.unsqueeze(1).to(device)
            
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            total_loss += loss.item()
            
            preds = (torch.sigmoid(outputs) > threshold).float()
            
            # ✅ CORRETTO: Calcola le metriche direttamente
            for i in range(preds.shape[0]):
                pred_single = preds[i].squeeze()
                mask_single = masks[i].squeeze()
                
                # Usa le funzioni statiche di LaneMetrics
                all_metrics['iou'].append(LaneMetrics.iou(pred_single, mask_single))
                all_metrics['dice'].append(LaneMetrics.dice_coefficient(pred_single, mask_single))
                all_metrics['f1'].append(LaneMetrics.f1_score(pred_single, mask_single))
    
    avg_loss = total_loss / len(val_loader)
    
    # Calcola le medie
    metrics_result = {
        'iou': np.mean(all_metrics['iou']) if all_metrics['iou'] else 0.0,
        'dice': np.mean(all_metrics['dice']) if all_metrics['dice'] else 0.0,
        'f1': np.mean(all_metrics['f1']) if all_metrics['f1'] else 0.0,
    }
    
    return avg_loss, metrics_result


# ==================== MAIN ====================

def main():
    config = Config()
    
    # Verifica paths
    if not os.path.exists(config.VAL_IMAGES_DIR):
        raise FileNotFoundError(f"❌ Cartella non trovata: {config.VAL_IMAGES_DIR}")
    
    if not os.path.exists(config.VAL_MASKS_DIR):
        raise FileNotFoundError(f"❌ Maschere non trovate: {config.VAL_MASKS_DIR}")
    
    if not os.path.exists(config.TEST_IMAGES_DIR):
        raise FileNotFoundError(f"❌ Cartella non trovata: {config.TEST_IMAGES_DIR}")
    
    if not os.path.exists(config.TEST_MASKS_DIR):
        raise FileNotFoundError(f"❌ Maschere non trovate: {config.TEST_MASKS_DIR}")
    
    os.makedirs(config.MODEL_DIR, exist_ok=True)
    
    print("="*80)
    print("🚀 TRAINING U-NET SU MOLANE")
    print("🎯 Dataset: MoLane val/target + test/target")
    print("🎯 Loss: Focal Tversky (gamma=0.75) + Dice")
    print("⚡ Mixed Precision: Enabled")
    print("="*80)
    print(f"Encoder: {config.ENCODER}")
    print(f"Device: {config.DEVICE}")
    print(f"Learning rate: {config.LEARNING_RATE} (with warmup)")
    print(f"Epochs: {config.EPOCHS}")
    print(f"Batch size: {config.BATCH_SIZE}")
    print("="*80)
    
    model = create_model(config)
    model.to(config.DEVICE)
    
    train_loader, val_loader = create_dataloaders_molane(config)
    
    loss_fn = CombinedLossOptimized(focal_weight=0.6, dice_weight=0.4)
    
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config.LEARNING_RATE,
        weight_decay=config.WEIGHT_DECAY
    )
    
    scheduler = WarmupCosineScheduler(
        optimizer,
        warmup_epochs=config.WARMUP_EPOCHS,
        max_epochs=config.EPOCHS,
        min_lr=1e-6
    )
    
    scaler = GradScaler() if config.USE_AMP else None
    
    best_iou = 0.0
    best_metrics = {}
    no_improve_count = 0
    patience = 10
    best_threshold = 0.5
    
    print("\n🏋️ Inizio training su MoLane...\n")
    
    for epoch in range(config.EPOCHS):
        print(f"\n{'='*80}")
        print(f"Epoch {epoch + 1}/{config.EPOCHS}")
        print(f"{'='*80}")
        
        current_lr = scheduler.step(epoch)
        
        train_loss = train_epoch(
            model, train_loader, optimizer, loss_fn,
            config.DEVICE, scaler, config.USE_AMP
        )
        
        val_loss, metrics = validate_epoch(
            model, val_loader, loss_fn, config.DEVICE, threshold=best_threshold
        )
        
        print(f"\n📈 RISULTATI VALIDAZIONE:")
        print(f" Train Loss: {train_loss:.4f}")
        print(f" Val Loss: {val_loss:.4f}")
        print(f" ───────────────────────")
        print(f" 🎯 IoU: {metrics['iou']:.4f}")
        print(f" F1: {metrics['f1']:.4f}")
        print(f" Dice: {metrics['dice']:.4f}")
        print(f" LR: {current_lr:.6f}")
        
        torch.save(model.state_dict(), config.LAST_MODEL_PATH)
        
        if metrics['iou'] > best_iou:
            best_iou = metrics['iou']
            best_metrics = metrics
            no_improve_count = 0
            torch.save(model.state_dict(), config.BEST_MODEL_PATH)
            print(f"\n ✅ NUOVO MIGLIOR MODELLO!")
            print(f" IoU: {best_iou:.4f}")
        else:
            no_improve_count += 1
            if no_improve_count >= patience:
                print(f"\n⚠️ Early stopping: nessun miglioramento per {patience} epoch")
                break
    
    print("\n" + "="*80)
    print("✅ TRAINING SU MOLANE COMPLETATO!")
    print("="*80)
    print(f"IoU: {best_iou:.4f} 🎯")
    print(f"F1: {best_metrics['f1']:.4f}")
    print(f"Modello: {config.BEST_MODEL_PATH}")
    print("="*80)


if __name__ == '__main__':
    main()


Overwriting train.py


In [84]:
%%writefile dataset.py
# dataset_MOLANE.py - Carica MoLane test/target + val/target

import os
import cv2
import numpy as np
from torch.utils.data import Dataset

class LaneSegmentationDataset(Dataset):
    """
    Dataset per MoLane che carica:
    - Validation set: /kaggle/input/carlane-benchmark/CARLANE/MoLane/data/val/target
    - Test set: /kaggle/input/carlane-benchmark/CARLANE/MoLane/data/test/target
    
    Con le maschere appena create:
    - /kaggle/working/molane_val_masks
    - /kaggle/working/molane_test_masks
    """
    
    def __init__(self, images_dir, masks_dir, transform=None, preprocessing=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.preprocessing = preprocessing
        self.img_size = (512, 512)
        
        # Carica RICORSIVAMENTE tutte le immagini dalle sottocartelle
        self.images_fps = []
        self.masks_fps = []
        
        for root, dirs, files in os.walk(images_dir):
            for file in sorted(files):
                if file.endswith(('.jpg', '.png', '.jpeg')):
                    img_path = os.path.join(root, file)
                    
                    # Mantieni il percorso relativo per trovare la maschera corrispondente
                    rel_path = os.path.relpath(img_path, images_dir)
                    
                    # Maschera corrisponde: stessa struttura, estensione .png
                    mask_rel_path = rel_path.replace('image', 'label').replace('.jpg', '.png')
                    mask_path = os.path.join(masks_dir, mask_rel_path)
                    
                    if os.path.exists(mask_path):
                        self.images_fps.append(img_path)
                        self.masks_fps.append(mask_path)
        
        print(f"✅ Dataset MoLane caricato: {len(self.images_fps)} immagini con maschere")
        
        assert len(self.images_fps) == len(self.masks_fps), \
            f"Numero immagini ({len(self.images_fps)}) != numero maschere ({len(self.masks_fps)})"
    
    def __len__(self):
        return len(self.images_fps)
    
    def __getitem__(self, idx):
        image = cv2.imread(self.images_fps[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        mask = cv2.imread(self.masks_fps[idx], cv2.IMREAD_GRAYSCALE)
        
        # ✅ RIDIMENSIONA PRIMA!
        image = cv2.resize(image, self.img_size, interpolation=cv2.INTER_LINEAR)
        mask = cv2.resize(mask, self.img_size, interpolation=cv2.INTER_NEAREST)
        
        mask = (mask > 127).astype(np.float32)
        
        # ✅ ORA le dimensioni sono uguali!
        if self.transform:
            sample = self.transform(image=image, mask=mask)
        
        return image, mask


Overwriting dataset.py


In [78]:
%%writefile augmentation.py
# augmentation_DEFINITIVA.py - BASATA DIRETTAMENTE SULLA DOCUMENTAZIONE UFFICIALE
# Tutti i parametri estratti e verificati dalla documentazione ufficiale di Albumentations
# Nessun parametro inventato o non verificato

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2


def get_training_augmentation_improved():
    """
    ✅ VERSIONE DEFINITIVA - PARAMETRI DALLA DOCUMENTAZIONE UFFICIALE
    Data augmentation per lane detection
    """

    train_transform = [
        # Resize base
        A.Resize(512, 512),

        # RandomResizedCrop - parametri corretti
        A.RandomResizedCrop(
            size=(512, 512),
            scale=(0.8, 1.0),
            ratio=(0.9, 1.1),
            interpolation=cv2.INTER_LINEAR,
            p=0.5
        ),

        # Flip orizzontale
        A.HorizontalFlip(p=0.5),

        # ShiftScaleRotate - parametri verificati
        A.ShiftScaleRotate(
            shift_limit=0.1,
            scale_limit=0.15,
            rotate_limit=15,
            interpolation=cv2.INTER_LINEAR,
            border_mode=cv2.BORDER_CONSTANT,
            fill=0,
            fill_mask=0,
            p=0.7
        ),

        # Perspective - parametri dalla documentazione ufficiale
        # scale: distortion scale (0.05, 0.1)
        # keep_size: resize back to original (True)
        # fit_output: adjust plane size (False)
        # border_mode: padding mode (cv2.BORDER_CONSTANT)
        # fill: padding value (0)
        # fill_mask: padding for mask (0)
        A.Perspective(
            scale=(0.05, 0.1),
            keep_size=True,
            fit_output=False,
            border_mode=cv2.BORDER_CONSTANT,
            fill=0,
            fill_mask=0,
            interpolation=cv2.INTER_LINEAR,
            p=0.3
        ),

        # RandomBrightnessContrast - parametri verificati
        A.RandomBrightnessContrast(
            brightness_limit=0.3,
            contrast_limit=0.3,
            p=0.5
        ),

        # CLAHE - parametri verificati
        A.CLAHE(
            clip_limit=2.0,
            tile_grid_size=(8, 8),
            p=0.4
        ),

        # GaussianBlur - parametri verificati
        A.GaussianBlur(
            blur_limit=(3, 5),
            p=0.2
        ),

        # CoarseDropout - parametri dalla documentazione ufficiale
        # num_holes_range: tuple (min, max) number of holes
        # hole_height_range: tuple (min, max) height range
        # hole_width_range: tuple (min, max) width range
        # fill: fill value ('random_uniform' per colore casuale)
        # fill_mask: mask fill value (None per non cambiare mask)
        A.CoarseDropout(
            num_holes_range=(1, 8),
            hole_height_range=(8, 32),
            hole_width_range=(8, 32),
            fill="random_uniform",
            fill_mask=0,
            p=0.3
        ),

        # RandomRain - parametri dalla documentazione ufficiale
        # slant_range: tuple (min, max) for slant angle
        # drop_length: int or None - length of drops (None = auto)
        # drop_width: int - width of drops
        # drop_color: tuple RGB
        # blur_value: int - blur amount
        # brightness_coefficient: float (0, 1] - brightness multiplier
        # rain_type: "drizzle", "heavy", "torrential", or "default"
        A.RandomRain(
            slant_range=(-10, 10),
            drop_length=20,
            drop_width=1,
            drop_color=(200, 200, 200),
            blur_value=3,
            brightness_coefficient=0.8,
            rain_type="drizzle",
            p=0.1
        ),

        # RandomFog - parametri dalla documentazione ufficiale
        # alpha_coef: float (0, 1] - transparency of fog
        # fog_coef_range: tuple (min, max) for fog intensity
        A.RandomFog(
            alpha_coef=0.08,
            fog_coef_range=(0.3, 1),
            p=0.1
        ),

        # Normalizzazione ImageNet
        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225),
            max_pixel_value=255.0,
        ),

        # Conversione a PyTorch format
        ToTensorV2(),
    ]

    return A.Compose(train_transform)


def get_validation_augmentation_improved():
    """
    Augmentation per validation - solo resize + normalize
    """

    val_transform = [
        A.Resize(512, 512),

        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225),
            max_pixel_value=255.0,
        ),

        ToTensorV2(),
    ]

    return A.Compose(val_transform)


def get_moderate_augmentation():
    """
    ✅ AUGMENTATION MODERATA
    Versione più leggera se quella aggressiva causa problemi
    """

    train_transform = [
        A.Resize(512, 512),

        A.HorizontalFlip(p=0.5),

        A.ShiftScaleRotate(
            shift_limit=0.0625,
            scale_limit=0.1,
            rotate_limit=10,
            border_mode=cv2.BORDER_CONSTANT,
            fill=0,
            fill_mask=0,
            p=0.6
        ),

        A.RandomBrightnessContrast(
            brightness_limit=0.2,
            contrast_limit=0.2,
            p=0.5
        ),

        A.CLAHE(
            clip_limit=2.0,
            tile_grid_size=(8, 8),
            p=0.3
        ),

        A.GaussianBlur(blur_limit=3, p=0.2),

        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225),
            max_pixel_value=255.0,
        ),

        ToTensorV2(),
    ]

    return A.Compose(train_transform)


def get_light_augmentation():
    """
    ✅ AUGMENTATION LEGGERA
    Solo trasformazioni essenziali
    """

    train_transform = [
        A.Resize(512, 512),

        A.HorizontalFlip(p=0.5),

        A.ShiftScaleRotate(
            shift_limit=0.05,
            scale_limit=0.1,
            rotate_limit=10,
            border_mode=cv2.BORDER_CONSTANT,
            fill=0,
            fill_mask=0,
            p=0.5
        ),

        A.RandomBrightnessContrast(
            brightness_limit=0.2,
            contrast_limit=0.2,
            p=0.5
        ),

        A.CLAHE(
            clip_limit=2.0,
            tile_grid_size=(8, 8),
            p=0.3
        ),

        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225),
            max_pixel_value=255.0,
        ),

        ToTensorV2(),
    ]

    return A.Compose(train_transform)


def get_test_time_augmentation():
    """
    ✅ Test Time Augmentation (TTA)
    Solo flip orizzontale - il più sicuro e efficace
    """

    tta_transforms = []

    # Original
    tta_transforms.append(A.Compose([
        A.Resize(512, 512),
        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225),
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]))

    # Horizontal flip
    tta_transforms.append(A.Compose([
        A.Resize(512, 512),
        A.HorizontalFlip(p=1.0),
        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225),
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]))

    return tta_transforms


# ==================== TESTING ====================
if __name__ == '__main__':
    import numpy as np

    print("✅ Test augmentation definitiva (parametri ufficiali)...\n")

    # Immagine e mask dummy
    dummy_image = np.random.randint(0, 255, (720, 1280, 3), dtype=np.uint8)
    dummy_mask = np.random.randint(0, 2, (720, 1280), dtype=np.float32)

    try:
        # Test training augmentation
        print("📊 Testing Training Augmentation (Full)...")
        train_aug = get_training_augmentation_improved()
        augmented = train_aug(image=dummy_image, mask=dummy_mask)

        print(f"   Input shape: {dummy_image.shape}")
        print(f"   Output image shape: {augmented['image'].shape}")
        print(f"   Output mask shape: {augmented['mask'].shape}")
        print(f"   Image dtype: {augmented['image'].dtype}")
        print(f"   Mask dtype: {augmented['mask'].dtype}")
        print(f"   ✅ Training augmentation OK!")

        # Test validation augmentation
        print("\n📊 Testing Validation Augmentation...")
        val_aug = get_validation_augmentation_improved()
        val_augmented = val_aug(image=dummy_image, mask=dummy_mask)
        print(f"   Output image shape: {val_augmented['image'].shape}")
        print(f"   Output mask shape: {val_augmented['mask'].shape}")
        print(f"   ✅ Validation augmentation OK!")

        # Test moderate augmentation
        print("\n📊 Testing Moderate Augmentation...")
        mod_aug = get_moderate_augmentation()
        mod_augmented = mod_aug(image=dummy_image, mask=dummy_mask)
        print(f"   ✅ Moderate augmentation OK!")

        # Test light augmentation
        print("\n📊 Testing Light Augmentation...")
        light_aug = get_light_augmentation()
        light_augmented = light_aug(image=dummy_image, mask=dummy_mask)
        print(f"   ✅ Light augmentation OK!")

        # Test TTA
        print("\n📊 Testing TTA...")
        tta_transforms = get_test_time_augmentation()
        print(f"   TTA transforms: {len(tta_transforms)}")
        print(f"   ✅ TTA OK!")

        print("\n" + "=" * 70)
        print("✅ TUTTI I TEST COMPLETATI CON SUCCESSO!")
        print("=" * 70)
        print("\n📝 COME USARE:")
        print("1. Per training: get_training_augmentation_improved()")
        print("2. Se overfitting: get_moderate_augmentation()")
        print("3. Se dataset piccolo: get_light_augmentation()")
        print("4. Per validation: get_validation_augmentation_improved()")
        print("5. Per TTA inference: get_test_time_augmentation()")
        print("\n✅ File pronto per essere usato in train.py!")

    except Exception as e:
        print(f"\n❌ ERRORE: {e}")
        import traceback

        traceback.print_exc()
        print("\nVerifica che albumentations sia aggiornato:")
        print("pip install -U albumentations")



Writing augmentation.py


In [80]:
%%writefile metrics.py
# metrics_CORRECTED.py - METRICHE CORRETTE PER LANE DETECTION

import torch
import numpy as np

class LaneMetrics:
    """Metriche specializzate per lane detection - VERSIONE CORRETTA"""
    
    # ==================== 1️⃣ DICE COEFFICIENT ====================
    @staticmethod
    def dice_coefficient(pred, target, smooth=1.0):
        """
        ✅ CORRETTO
        Formula: Dice = (2 * TP) / (2 * TP + FP + FN)
        """
        # Converti a binario (pred potrebbe già essere binario o probabilità)
        if pred.max() > 1.0:
            pred_binary = (pred > 0.5).float()
        else:
            pred_binary = pred
        
        target = target.float()
        
        intersection = (pred_binary * target).sum()
        dice = (2.0 * intersection + smooth) / (pred_binary.sum() + target.sum() + smooth)
        return dice.item()
    
    
    # ==================== 2️⃣ SENSITIVITY (RECALL) ====================
    @staticmethod
    def sensitivity(pred, target, smooth=1e-6):
        """
        ✅ CORRETTO
        Formula: Sensitivity = TP / (TP + FN)
        
        ⚠️ BUG ORIGINALE: Non convertiva pred a binario!
        """
        # ✅ FIX: Converti a binario
        if pred.max() > 1.0:
            pred_binary = (pred > 0.5).float()
        else:
            pred_binary = pred
        
        target = target.float()
        
        TP = (pred_binary * target).sum()
        FN = ((1 - pred_binary) * target).sum()
        
        sensitivity = TP / (TP + FN + smooth)
        return sensitivity.item()
    
    
    # ==================== 3️⃣ SPECIFICITY ====================
    @staticmethod
    def specificity(pred, target, smooth=1e-6):
        """
        ✅ CORRETTO
        Formula: Specificity = TN / (TN + FP)
        
        ⚠️ BUG ORIGINALE: Non convertiva pred a binario!
        """
        # ✅ FIX: Converti a binario
        if pred.max() > 1.0:
            pred_binary = (pred > 0.5).float()
        else:
            pred_binary = pred
        
        target = target.float()
        
        TN = ((1 - pred_binary) * (1 - target)).sum()
        FP = (pred_binary * (1 - target)).sum()
        
        specificity = TN / (TN + FP + smooth)
        return specificity.item()
    
    
    # ==================== 4️⃣ F1 SCORE ====================
    @staticmethod
    def f1_score(pred, target, smooth=1e-6):
        """
        ✅ CORRETTO
        Formula: F1 = 2 * (Precision * Recall) / (Precision + Recall)
        """
        # ✅ FIX: Converti a binario
        if pred.max() > 1.0:
            pred_binary = (pred > 0.5).float()
        else:
            pred_binary = pred
        
        target = target.float()
        
        TP = (pred_binary * target).sum()
        FP = (pred_binary * (1 - target)).sum()
        FN = ((1 - pred_binary) * target).sum()
        
        precision = TP / (TP + FP + smooth)
        recall = TP / (TP + FN + smooth)
        f1 = 2 * (precision * recall) / (precision + recall + smooth)
        
        return f1.item()
    
    
    # ==================== 5️⃣ MATTHEWS CORRELATION COEFFICIENT ====================
    @staticmethod
    def mcc(pred, target, smooth=1e-6):
        """
        ✅ CORRETTO
        Formula: MCC = (TP*TN - FP*FN) / sqrt((TP+FP)(TP+FN)(TN+FP)(TN+FN))
        
        ⚠️ BUG ORIGINALE: Non gestiva correttamente i tensori
        """
        # ✅ FIX: Converti a binario
        if pred.max() > 1.0:
            pred_binary = (pred > 0.5).float()
        else:
            pred_binary = pred
        
        target = target.float()
        
        TP = (pred_binary * target).sum()
        TN = ((1 - pred_binary) * (1 - target)).sum()
        FP = (pred_binary * (1 - target)).sum()
        FN = ((1 - pred_binary) * target).sum()
        
        numerator = TP * TN - FP * FN
        denominator = torch.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN) + smooth)
        
        # ✅ FIX: Gestisci il caso di denominatore zero
        if denominator == 0:
            return 0.0
        
        mcc = numerator / denominator
        return mcc.item()
    
    
    # ==================== 6️⃣ PIXEL ACCURACY ====================
    @staticmethod
    def pixel_accuracy(pred, target, smooth=1e-6):
        """
        ✅ CORRETTO
        Formula: Accuracy = (TP + TN) / Total
        """
        # ✅ FIX: Converti a binario
        if pred.max() > 1.0:
            pred_binary = (pred > 0.5).float()
        else:
            pred_binary = pred
        
        target = target.float()
        
        correct = (pred_binary == target).float().sum()
        total = target.numel()
        
        # ✅ FIX: Gestisci il caso di total zero
        if total == 0:
            return 0.0
        
        return (correct / total).item()
    
    
    # ==================== 7️⃣ IoU (Intersection over Union) ====================
    @staticmethod
    def iou(pred, target, smooth=1e-6):
        """
        ✅ AGGIUNTO - Lo standard per segmentazione
        Formula: IoU = TP / (TP + FP + FN)
        """
        # ✅ FIX: Converti a binario
        if pred.max() > 1.0:
            pred_binary = (pred > 0.5).float()
        else:
            pred_binary = pred
        
        target = target.float()
        
        intersection = (pred_binary * target).sum()
        union = pred_binary.sum() + target.sum() - intersection
        
        iou_score = (intersection + smooth) / (union + smooth)
        return iou_score.item()


# ==================== FUNZIONE DI UTILITÀ ====================

def calculate_all_metrics(pred_batch, target_batch):
    """
    ✅ CORRETTA - Calcola tutte le metriche per un batch
    
    Input:
        pred_batch: tensor [B, H, W] con probabilità [0, 1] o valori > 1
        target_batch: tensor [B, H, W] con valori binari {0, 1}
    
    Output:
        dict con tutte le metriche
    """
    metrics = {
        'iou': [],
        'dice': [],
        'sensitivity': [],
        'specificity': [],
        'f1': [],
        'mcc': [],
        'accuracy': [],
    }
    
    # ✅ FIX: Itera correttamente su ogni elemento del batch
    for batch_idx in range(pred_batch.shape[0]):
        pred_single = pred_batch[batch_idx]    # [H, W]
        target_single = target_batch[batch_idx] # [H, W]
        
        # Calcola tutte le metriche per questa immagine
        metrics['iou'].append(LaneMetrics.iou(pred_single, target_single))
        metrics['dice'].append(LaneMetrics.dice_coefficient(pred_single, target_single))
        metrics['sensitivity'].append(LaneMetrics.sensitivity(pred_single, target_single))
        metrics['specificity'].append(LaneMetrics.specificity(pred_single, target_single))
        metrics['f1'].append(LaneMetrics.f1_score(pred_single, target_single))
        metrics['mcc'].append(LaneMetrics.mcc(pred_single, target_single))
        metrics['accuracy'].append(LaneMetrics.pixel_accuracy(pred_single, target_single))
    
    # ✅ Restituisci media di tutte le metriche
    return {k: np.mean(v) if v else 0.0 for k, v in metrics.items()}


# ==================== DEBUG ====================

if __name__ == '__main__':
    print("✅ Test metriche corrette...\n")
    
    # Crea dati fittizi
    pred = torch.rand(256, 256)           # Probabilità [0, 1]
    target = torch.randint(0, 2, (256, 256)).float()  # Binario
    
    print("📊 Metriche per Lane Detection:\n")
    print(f"  IoU:              {LaneMetrics.iou(pred, target):.4f}")
    print(f"  Dice:             {LaneMetrics.dice_coefficient(pred, target):.4f}")
    print(f"  Sensitivity:      {LaneMetrics.sensitivity(pred, target):.4f} ← Corsie trovate")
    print(f"  Specificity:      {LaneMetrics.specificity(pred, target):.4f} ← Falsi positivi")
    print(f"  F1 Score:         {LaneMetrics.f1_score(pred, target):.4f}")
    print(f"  MCC:              {LaneMetrics.mcc(pred, target):.4f}")
    print(f"  Accuracy:         {LaneMetrics.pixel_accuracy(pred, target):.4f}")
    
    # Test batch
    print("\n\n📊 Test Batch:\n")
    pred_batch = torch.rand(4, 256, 256)
    target_batch = torch.randint(0, 2, (4, 256, 256)).float()
    
    all_metrics = calculate_all_metrics(pred_batch, target_batch)
    for k, v in all_metrics.items():
        print(f"  {k.upper():12s}: {v:.4f}")


Writing metrics.py


In [11]:
!pip install segmentation_models_pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 4.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: n

In [ ]:
!python train.py

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [68]:
%%writefile adaptation.py
# ============================================
# FINE-TUNING CON MOLANE LABELS - VERSIONE CORRETTA
# ============================================

import torch
import torch.nn as nn
import torch.optim as optim
import os
import cv2
import numpy as np
from torch.utils.data import DataLoader, Dataset, ConcatDataset
import segmentation_models_pytorch as smp
from tqdm import tqdm
import gc

# ============================================
# LOSS FUNCTIONS
# ============================================

class FocalTverskyLoss(nn.Module):
    def __init__(self, alpha=0.3, beta=0.7, gamma=0.75, smooth=1.0):
        super().__init__()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.smooth = smooth
    
    def forward(self, predictions, targets):
        probs = torch.sigmoid(predictions)
        TP = (probs * targets).sum()
        FP = (probs * (1 - targets)).sum()
        FN = ((1 - probs) * targets).sum()
        
        tversky_index = (TP + self.smooth) / (
            TP + self.alpha * FP + self.beta * FN + self.smooth
        )
        
        focal_tversky = torch.pow(1 - tversky_index, self.gamma)
        return focal_tversky


class CombinedLossOptimized(nn.Module):
    def __init__(self, focal_weight=0.6, dice_weight=0.4):
        super().__init__()
        self.focal_weight = focal_weight
        self.dice_weight = dice_weight
        self.focal_tversky = FocalTverskyLoss(alpha=0.3, beta=0.7, gamma=0.75)
    
    def forward(self, predictions, targets):
        focal = self.focal_tversky(predictions, targets)
        
        probs = torch.sigmoid(predictions)
        intersection = (probs * targets).sum()
        dice = (2.0 * intersection + 1.0) / (probs.sum() + targets.sum() + 1.0)
        dice_loss = 1 - dice
        
        return self.focal_weight * focal + self.dice_weight * dice_loss


# ============================================
# UTILITIES
# ============================================

def cleanup_memory():
    gc.collect()
    torch.cuda.empty_cache()


# ============================================
# DATASET
# ============================================

class MoLaneLabeledDataset(Dataset):
    def __init__(self, images_base_dir, masks_base_dir, img_size=(512, 512)):
        self.img_size = img_size
        self.images_base_dir = images_base_dir
        self.masks_base_dir = masks_base_dir
        self.image_files = []
        
        print(f"🔍 Searching for labeled samples...")
        
        for root, dirs, files in os.walk(images_base_dir):
            for file in files:
                if file.endswith(('.jpg', '.png')):
                    rel_path = os.path.relpath(os.path.join(root, file), images_base_dir)
                    self.image_files.append(rel_path)
        
        self.image_files = sorted(self.image_files)
        print(f"✅ Found {len(self.image_files)} labeled samples\n")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_rel_path = self.image_files[idx]
        
        img_path = os.path.join(self.images_base_dir, img_rel_path)
        img = cv2.imread(img_path)
        
        if img is None:
            return self.__getitem__(0)
        
        # 🔧 CORREZIONE: Sostituisci "image" con "label" E ".jpg" con ".png"
        mask_rel_path = img_rel_path.replace('image', 'label').replace('.jpg', '.png')
        mask_path = os.path.join(self.masks_base_dir, mask_rel_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        if mask is None:
            print(f"⚠️ Could not load mask: {mask_path}")
            return self.__getitem__(0)
        
        img = cv2.resize(img, self.img_size)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.0
        img = np.transpose(img, (2, 0, 1))
        
        mask = cv2.resize(mask, self.img_size, interpolation=cv2.INTER_NEAREST)
        mask = mask.astype(np.float32) / 255.0
        
        return torch.from_numpy(img).float(), torch.from_numpy(mask).float()



# ============================================
# TRAINING
# ============================================

def train_epoch(model, train_loader, loss_fn, optimizer, device, epoch, total_epochs):
    model.train()
    total_loss = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{total_epochs}")
    
    for images, masks in pbar:
        images = images.to(device)
        masks = masks.unsqueeze(1).to(device)
        
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
        optimizer.step()
        
        total_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.6f}'})  # ← CORRETTO!
    
    avg_loss = total_loss / len(train_loader)
    return avg_loss


def validate(model, val_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for images, masks in tqdm(val_loader, desc="Validating"):
            images = images.to(device)
            masks = masks.unsqueeze(1).to(device)
            
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            total_loss += loss.item()
    
    avg_loss = total_loss / len(val_loader)
    return avg_loss


# ============================================
# MAIN
# ============================================

def main():
    print("\n" + "="*80)
    print("🚀 FINE-TUNING ON MOLANE LABELED DATA")
    print("="*80 + "\n")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}\n")
    
    cleanup_memory()
    
    # ============ STEP 1: Carica Dataset ============
    print("STEP 1: Loading labeled datasets...")
    print("-" * 60)
    
    val_images = '/kaggle/input/carlane-benchmark/CARLANE/MoLane/data/val/target'
    val_masks = '/kaggle/working/molane_val_masks'
    
    try:
        val_dataset = MoLaneLabeledDataset(val_images, val_masks)
        val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False, num_workers=1)
    except Exception as e:
        print(f"❌ Error loading validation set: {e}")
        return
    
    test_images = '/kaggle/input/carlane-benchmark/CARLANE/MoLane/data/test/target'
    test_masks = '/kaggle/working/molane_test_masks'
    
    try:
        test_dataset = MoLaneLabeledDataset(test_images, test_masks)
        test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=1)
    except Exception as e:
        print(f"❌ Error loading test set: {e}")
        test_dataset = None
    
    if test_dataset:
        combined_dataset = ConcatDataset([val_dataset, test_dataset])
        print(f"✅ Validation set: {len(val_dataset)} samples")
        print(f"✅ Test set: {len(test_dataset)} samples")
    else:
        combined_dataset = val_dataset
        print(f"✅ Validation set: {len(val_dataset)} samples")
    
    train_loader = DataLoader(combined_dataset, batch_size=10, shuffle=True, num_workers=1)
    print(f"✅ Combined: {len(combined_dataset)} samples\n")
    
    # ============ STEP 2: Carica Modello ============
    print("STEP 2: Loading pre-trained model...")
    print("-" * 60)
    
    print("📂 Loading model...")
    
    model = smp.Unet(
        encoder_name='efficientnet-b4',
        encoder_weights='imagenet',
        classes=1,
        activation=None,
        decoder_dropout=0.2,
    )
    
    model_path = '/kaggle/input/best-unet/pytorch/default/1/best_unet_improved.pth'
    checkpoint = torch.load(model_path, map_location=device)
    
    if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint)
    
    model = model.to(device)
    print("✅ Model loaded\n")
    
    # ============ STEP 3: Setup Training ============
    print("STEP 3: Setting up training...")
    print("-" * 60)
    
    loss_fn = CombinedLossOptimized(focal_weight=0.6, dice_weight=0.4)
    optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    
    print(f"Loss: Focal Tversky (0.6) + Dice (0.4)")
    print(f"Optimizer: AdamW (lr=1e-5)")
    print(f"Scheduler: CosineAnnealing\n")
    
    # ============ STEP 4: Training Loop ============
    print("STEP 4: Training...")
    print("-" * 60 + "\n")
    
    num_epochs = 10
    best_loss = float('inf')
    patience = 3
    patience_counter = 0
    
    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, loss_fn, optimizer, 
                                device, epoch, num_epochs)
        val_loss = validate(model, val_loader, loss_fn, device)
        
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss:.6f}")
        print(f"  Val Loss: {val_loss:.6f}")
        
        scheduler.step()
        
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
            print(f"  ✅ Best validation loss!")
        else:
            patience_counter += 1
            print(f"  ⚠️ No improvement ({patience_counter}/{patience})")
        
        if patience_counter >= patience:
            print(f"\n⛔ Early stopping after {epoch+1} epochs")
            break
        
        print()
    
    # ============ STEP 5: Salva Modello ============
    print("STEP 5: Saving model...")
    print("-" * 60)
    
    output_dir = '/kaggle/working'
    os.makedirs(output_dir, exist_ok=True)
    
    weights_path = os.path.join(output_dir, 'unet_finetuned_molane_weights.pth')
    torch.save(model.state_dict(), weights_path)
    
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'adaptation_method': 'Fine-tuning on MoLane labeled data',
        'loss_function': 'Focal Tversky (0.6) + Dice (0.4)',
        'timestamp': str(__import__('datetime').datetime.now())
    }
    
    checkpoint_path = os.path.join(output_dir, 'unet_finetuned_molane_checkpoint.pth')
    torch.save(checkpoint, checkpoint_path)
    
    print(f"✅ Weights saved to: {weights_path}")
    print(f"✅ Checkpoint saved to: {checkpoint_path}\n")
    
    # ============ FINALE ============
    print("="*60)
    print("✨ FINE-TUNING COMPLETED!")
    print("="*60)
    print(f"\n📊 Summary:")
    print(f"   • Training samples: {len(combined_dataset)}")
    print(f"   • Best validation loss: {best_loss:.6f}")
    print(f"   • Total epochs: {epoch+1}")
    print(f"   • Model: smp.Unet(efficientnet-b4)")
    print(f"   • Loss: Focal Tversky + Dice")
    print(f"\n🎉 Model ready for deployment on your robot!")


if __name__ == "__main__":
    main()


Overwriting adaptation.py


In [69]:
!python adaptation.py

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [49]:
%%writefile conv.py
# ============================================
# SOLUTION: Copy existing labels directly
# ============================================

import os
import cv2
import shutil
from pathlib import Path

print("\n" + "="*80)
print("📋 COPYING EXISTING LABELS DIRECTLY")
print("="*80 + "\n")

molane_base = '/kaggle/input/carlane-benchmark/CARLANE/MoLane'
data_dir = os.path.join(molane_base, 'data')

# ============ VAL SET ============
print("🔷 VALIDATION SET")
print("="*80)

val_source = os.path.join(data_dir, 'val', 'target')
val_output = '/kaggle/working/molane_val_masks'

os.makedirs(val_output, exist_ok=True)

# Copia TUTTI i file .png da val/target (mantenendo struttura)
val_count = 0
for root, dirs, files in os.walk(val_source):
    for file in files:
        if file.endswith('.png'):
            # Mantieni la struttura delle subdirectory
            rel_path = os.path.relpath(os.path.join(root, file), val_source)
            output_subdir = os.path.dirname(os.path.join(val_output, rel_path))
            os.makedirs(output_subdir, exist_ok=True)
            
            src_path = os.path.join(root, file)
            dst_path = os.path.join(val_output, rel_path)
            
            # Ridimensiona mentre copi
            img = cv2.imread(src_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img_resized = cv2.resize(img, (512, 512), interpolation=cv2.INTER_NEAREST)
                cv2.imwrite(dst_path, img_resized)
                val_count += 1

print(f"✅ Copied {val_count} validation masks")

# ============ TEST SET ============
print("\n🔶 TEST SET")
print("="*80)

test_source = os.path.join(data_dir, 'test', 'target')
test_output = '/kaggle/working/molane_test_masks'

os.makedirs(test_output, exist_ok=True)

# Copia TUTTI i file .png da test/target
test_count = 0
for root, dirs, files in os.walk(test_source):
    for file in files:
        if file.endswith('.png'):
            rel_path = os.path.relpath(os.path.join(root, file), test_source)
            output_subdir = os.path.dirname(os.path.join(test_output, rel_path))
            os.makedirs(output_subdir, exist_ok=True)
            
            src_path = os.path.join(root, file)
            dst_path = os.path.join(test_output, rel_path)
            
            # Ridimensiona mentre copi
            img = cv2.imread(src_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img_resized = cv2.resize(img, (512, 512), interpolation=cv2.INTER_NEAREST)
                cv2.imwrite(dst_path, img_resized)
                test_count += 1

print(f"✅ Copied {test_count} test masks")

# ============ VERIFICA ============
print("\n" + "="*80)
print("✨ FINAL RESULTS")
print("="*80)
print(f"\n✅ Validation masks: {val_count}")
print(f"✅ Test masks: {test_count}")
print(f"✅ Total: {val_count + test_count}")

print(f"\n📁 Output locations:")
print(f"   • /kaggle/working/molane_val_masks ({val_count} files)")
print(f"   • /kaggle/working/molane_test_masks ({test_count} files)")

print(f"\n🎉 Ready for fine-tuning!\n")

# Verifica che i file siano stati salvati
if val_count > 0 and test_count > 0:
    print("✓ SUCCESS! Masks are ready to use")
else:
    print("❌ FAILED! Check the output directories")


Overwriting conv.py


In [70]:
%%writefile rescale_conv.py
# ============================================
# FIX: Riscala i valori delle maschere a 0-255
# ============================================

import os
import cv2
import numpy as np
from tqdm import tqdm

print("\n" + "="*80)
print("🔧 FIXING MASK VALUES")
print("="*80 + "\n")

def fix_mask_values(mask_dir):
    """Riscala i valori delle maschere da [0,2] a [0,255]"""
    
    fixed_count = 0
    
    for root, dirs, files in os.walk(mask_dir):
        for file in tqdm(files):
            if file.endswith('.png'):
                mask_path = os.path.join(root, file)
                
                # Leggi la maschera
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                
                if mask is None:
                    continue
                
                # Riscala i valori
                # Se max è 2, moltiplica per 127.5 per portare a 0-255
                if mask.max() > 0:
                    mask_fixed = np.uint8((mask / mask.max()) * 255)
                else:
                    mask_fixed = mask.astype(np.uint8)
                
                # Salva la maschera corretta
                cv2.imwrite(mask_path, mask_fixed)
                fixed_count += 1
    
    return fixed_count

# ============ FIX VALIDATION ============
print("🔷 Fixing validation masks...")
val_masks = '/kaggle/working/molane_val_masks'
val_fixed = fix_mask_values(val_masks)
print(f"✅ Fixed {val_fixed} validation masks\n")

# ============ FIX TEST ============
print("🔶 Fixing test masks...")
test_masks = '/kaggle/working/molane_test_masks'
test_fixed = fix_mask_values(test_masks)
print(f"✅ Fixed {test_fixed} test masks\n")

# ============ VERIFICA ============
print("="*80)
print("🔍 VERIFICATION")
print("="*80 + "\n")

# Verifica il primo file di ogni set
for label, mask_dir in [("Validation", val_masks), ("Test", test_masks)]:
    for root, dirs, files in os.walk(mask_dir):
        for file in files:
            if file.endswith('.png'):
                test_path = os.path.join(root, file)
                mask = cv2.imread(test_path, cv2.IMREAD_GRAYSCALE)
                
                print(f"{label} sample: {os.path.relpath(test_path, mask_dir)}")
                print(f"  Shape: {mask.shape}")
                print(f"  Min: {mask.min()}")
                print(f"  Max: {mask.max()}")
                print(f"  Mean: {mask.mean():.2f}")
                print(f"  Lane pixels (255): {(mask == 255).sum()}")
                print(f"  Background pixels (0): {(mask == 0).sum()}\n")
                break
        else:
            continue
        break

print("="*80)
print("✨ MASKS FIXED AND READY!")
print("="*80 + "\n")


Writing rescale_conv.py


In [51]:
import cv2
import numpy as np

# Leggi una maschera
mask = cv2.imread('/kaggle/working/molane_val_masks/roundtrack/black/left_curve/dark/0001_label.png', 
                   cv2.IMREAD_GRAYSCALE)

print(f"Mask shape: {mask.shape}")
print(f"Min value: {mask.min()}")
print(f"Max value: {mask.max()}")
print(f"Mean value: {mask.mean():.2f}")
print(f"Pixels with lane (255): {(mask == 255).sum()}")
print(f"Pixels without lane (0): {(mask == 0).sum()}")


Mask shape: (512, 512)
Min value: 0
Max value: 2
Mean value: 0.04
Pixels with lane (255): 0
Pixels without lane (0): 255315


In [50]:
!python conv.py


📋 COPYING EXISTING LABELS DIRECTLY

🔷 VALIDATION SET
✅ Copied 2000 validation masks

🔶 TEST SET
✅ Copied 1000 test masks

✨ FINAL RESULTS

✅ Validation masks: 2000
✅ Test masks: 1000
✅ Total: 3000

📁 Output locations:
   • /kaggle/working/molane_val_masks (2000 files)
   • /kaggle/working/molane_test_masks (1000 files)

🎉 Ready for fine-tuning!

✓ SUCCESS! Masks are ready to use


In [53]:
# ============================================
# COMPRESS AND DOWNLOAD
# ============================================

import os
import zipfile
import shutil

print("\n" + "="*80)
print("📦 COMPRESSING MASKS FOR DOWNLOAD")
print("="*80 + "\n")

# Crea una cartella ZIP
output_dir = '/kaggle/working'
zip_path = os.path.join(output_dir, 'molane_masks.zip')

print(f"Creating ZIP file: {zip_path}\n")

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Aggiungi validation masks
    val_dir = os.path.join(output_dir, 'molane_val_masks')
    for root, dirs, files in os.walk(val_dir):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, output_dir)
            zipf.write(file_path, arcname)
            print(f"  ✓ Added {arcname}")
    
    # Aggiungi test masks
    test_dir = os.path.join(output_dir, 'molane_test_masks')
    for root, dirs, files in os.walk(test_dir):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, output_dir)
            zipf.write(file_path, arcname)
            print(f"  ✓ Added {arcname}")

# Verifica il file
zip_size = os.path.getsize(zip_path) / 1e9
print(f"\n✅ ZIP created!")
print(f"   Size: {zip_size:.2f} GB")
print(f"   Path: {zip_path}")

print("\n📥 DOWNLOAD:")
print("   Go to 'Output' tab and download 'molane_masks.zip'")



📦 COMPRESSING MASKS FOR DOWNLOAD

Creating ZIP file: /kaggle/working/molane_masks.zip

  ✓ Added molane_val_masks/roundtrack/black/right_curve/light/0068_label.png
  ✓ Added molane_val_masks/roundtrack/black/right_curve/light/0124_label.png
  ✓ Added molane_val_masks/roundtrack/black/right_curve/light/0123_label.png
  ✓ Added molane_val_masks/roundtrack/black/right_curve/light/0086_label.png
  ✓ Added molane_val_masks/roundtrack/black/right_curve/light/0025_label.png
  ✓ Added molane_val_masks/roundtrack/black/right_curve/light/0014_label.png
  ✓ Added molane_val_masks/roundtrack/black/right_curve/light/0094_label.png
  ✓ Added molane_val_masks/roundtrack/black/right_curve/light/0041_label.png
  ✓ Added molane_val_masks/roundtrack/black/right_curve/light/0001_label.png
  ✓ Added molane_val_masks/roundtrack/black/right_curve/light/0097_label.png
  ✓ Added molane_val_masks/roundtrack/black/right_curve/light/0112_label.png
  ✓ Added molane_val_masks/roundtrack/black/right_curve/light/006